In [1]:
# To run this Notebook you need to clone the Bmad-X repository and install it
# as edditable!

In [2]:
import ARESlatticeStage3v1_9 as ares
import bmadx
import cheetah
import matplotlib.pyplot as plt
import ocelot
import torch
from ocelot.adaptors.astra2ocelot import astraBeam2particleArray

[INFO    ] csr.py: module PYFFTW is not installed. Install it to speed up calculation.


initializing ocelot...
import: module PYFFTW is not installed. Install it to speed up calculation


In [3]:
segment = cheetah.Segment.from_ocelot(ares.cell, warnings=False).subcell(
    "AREASOLA1", "AREABSCR1"
)
segment

Segment(elements=ModuleList(
  (0): Marker(name='AREASOLA1')
  (1): Drift(length=tensor(0.1750))
  (2): Quadrupole(length=tensor(0.1220), k1=tensor(0.), misalignment=tensor([0., 0.]), tilt=tensor(0.), name='AREAMQZM1')
  (3): Drift(length=tensor(0.4280))
  (4): Quadrupole(length=tensor(0.1220), k1=tensor(0.), misalignment=tensor([0., 0.]), tilt=tensor(0.), name='AREAMQZM2')
  (5): Drift(length=tensor(0.2040))
  (6): VerticalCorrector(length=tensor(0.0200), angle=tensor(0.), name='AREAMCVM1')
  (7): Drift(length=tensor(0.2040))
  (8): Quadrupole(length=tensor(0.1220), k1=tensor(0.), misalignment=tensor([0., 0.]), tilt=tensor(0.), name='AREAMQZM3')
  (9): Drift(length=tensor(0.1790))
  (10): HorizontalCorrector(length=tensor(0.0200), angle=tensor(0.), name='AREAMCHM1')
  (11): Drift(length=tensor(0.4500))
  (12): Screen(resolution=tensor([2448, 2040]), pixel_size=tensor([3.5488e-06, 2.5003e-06]), binning=tensor(1), misalignment=tensor([0., 0.]), is_active=False, name='AREABSCR1')
), name

In [4]:
lattice = [
    bmadx.Drift(L=0.1750),
    bmadx.Quadrupole(L=0.1220, K1=10.0),  # AREAMQZM1
    bmadx.Drift(L=0.4280),
    bmadx.Quadrupole(L=0.1220, K1=-9.0),  # AREAMQZM2
    bmadx.Drift(L=0.2040),
    bmadx.SBend(L=0.0200, G=1e-3 / 0.0200),  # AREAMCVM1
    bmadx.Drift(L=0.2040),
    bmadx.Quadrupole(L=0.1220, K1=-8.0),  # AREAMQZM3
    bmadx.Drift(L=0.1790),
    bmadx.SBend(L=0.0200, G=-2e-3 / 0.0200),  # AREAMCHM1
    bmadx.Drift(L=0.4500),
]

In [5]:
import numpy as np

np.random.multivariate_normal(
    mean=np.zeros(6), cov=1e-6 * np.identity(6), size=100_000
).shape

(100000, 6)

In [6]:
particle_array = astraBeam2particleArray("ACHIP_EA1_2021.1351.001")
beam = bmadx.Particle(
    *particle_array.rparticles, 0.0, particle_array.p0c * 1e9, bmadx.M_ELECTRON
)
beam

Astra to Ocelot: charge =  5.000000000001694e-13  C
Astra to Ocelot: particles number =  100000
Astra to Ocelot: energy =  0.10731590244355117  GeV
Astra to Ocelot: s pos =  13.50890653269  m


Particle(x=array([ 5.08957129e-08,  6.15034578e-05,  2.30254514e-08, ...,
        2.08621251e-04,  5.64747744e-05, -6.26614324e-05]), px=array([ 5.99166607e-09,  6.28096994e-07,  6.38793974e-09, ...,
        5.50625646e-06,  1.21757376e-06, -2.37840015e-06]), y=array([-5.47305446e-07, -5.50215348e-07,  4.40441298e-05, ...,
        2.01889701e-04,  2.77884048e-04,  2.16434721e-04]), py=array([-1.22334631e-08, -1.17054384e-08,  1.65533365e-07, ...,
        6.02466623e-06,  7.84711679e-06,  6.54106916e-06]), z=array([-0.00000000e+00,  2.18696679e-06,  6.85092638e-07, ...,
        8.31485105e-07,  1.18904531e-06,  6.57932034e-06]), pz=array([ 0.        , -0.00102566,  0.0011856 , ..., -0.00054731,
        0.00143682, -0.00181577]), s=0.0, p0c=107314685.84229998, mc2=510998.94999999995)

In [7]:
%%timeit
out_beam = bmadx.track_lattice(beam, lattice)

65.3 ms ± 32.7 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [8]:
out_beam = bmadx.track_lattice(beam, lattice)
out_beam

Particle(x=array([nan, nan, nan, ..., nan, nan, nan]), px=array([nan, nan, nan, ..., nan, nan, nan]), y=array([nan, nan, nan, ..., nan, nan, nan]), py=array([nan, nan, nan, ..., nan, nan, nan]), z=array([nan, nan, nan, ..., nan, nan, nan]), pz=array([nan, nan, nan, ..., nan, nan, nan]), s=2.046, p0c=array(nan), mc2=510998.94999999995)